In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
print(tf.__version__)

2.12.0


In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
fid = drive.ListFile({'q':"title='Final_datateset_for_Image.zip'"}).GetList()[0]['id']
f = drive.CreateFile({'id': fid})
f.GetContentFile('Final_datateset_for_Image.zip')
f.keys()
!unzip Final_datateset_for_Image.zip

Streaming output truncated to the last 5000 lines.
  inflating: Final datateset for Image/validation/happy/16037.jpg  
  inflating: Final datateset for Image/validation/happy/16042.jpg  
  inflating: Final datateset for Image/validation/happy/16064.jpg  
  inflating: Final datateset for Image/validation/happy/16076.jpg  
  inflating: Final datateset for Image/validation/happy/16081.jpg  
  inflating: Final datateset for Image/validation/happy/16106.jpg  
  inflating: Final datateset for Image/validation/happy/16108.jpg  
  inflating: Final datateset for Image/validation/happy/16120.jpg  
  inflating: Final datateset for Image/validation/happy/16141.jpg  
  inflating: Final datateset for Image/validation/happy/16192.jpg  
  inflating: Final datateset for Image/validation/happy/16198.jpg  
  inflating: Final datateset for Image/validation/happy/16238.jpg  
  inflating: Final datateset for Image/validation/happy/16275.jpg  
  inflating: Final datateset for Image/validation/happy/1628.jpg 

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D,Flatten,Dense,MaxPool2D,BatchNormalization,GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import preprocess_input,decode_predictions
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.models import Model
from keras.layers.serialization import activation
import numpy as np
import random as rn

In [ ]:
train_dir = '/content/Final datateset for Image/train'  
validation_dir = '/content/Final datateset for Image/validation'

In [ ]:
BATCH_SIZE = 50
IMG_SIZE = (224, 224)
no_classes = 13

In [ ]:
train_dataset = tf.keras.utils.image_dataset_from_directory(train_dir,shuffle=True,batch_size=BATCH_SIZE,image_size=IMG_SIZE)

validation_dataset = tf.keras.utils.image_dataset_from_directory(validation_dir,shuffle=True,batch_size=BATCH_SIZE,image_size=IMG_SIZE)

Found 29183 files belonging to 13 classes.
Found 7358 files belonging to 13 classes.


In [ ]:
class_names = train_dataset.class_names
print(class_names)

['angry', 'disgust', 'fear', 'happy', 'joiariya', 'meghna', 'neutral', 'nitul', 'rafat', 'rasa', 'romiz', 'sad', 'surprise']


In [ ]:
#visulized some random Image With their Label
plt.figure(figsize=(10, 10))
for images, labels in train_dataset.take(1):
  for i in range(12):
    ax = plt.subplot(5, 4, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
# preprocessing

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   validation_split=0.4)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(224,224),
                                                    batch_size=50,
                                                    class_mode ='categorical',
                                                    subset='training')

Found 17514 images belonging to 13 classes.


In [ ]:
test_generator = train_datagen.flow_from_directory(validation_dir,
                                                   target_size=(224,224),
                                                   batch_size=50,
                                                   class_mode='categorical',
                                                   subset='validation')

Found 2939 images belonging to 13 classes.


In [ ]:
# Build  RESNET50 Model
num_classes = 13

base_model = ResNet50(include_top= False, weights= None)

x= base_model.output
x= GlobalAveragePooling2D()(x)
x= Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs =base_model.input,outputs=predictions)

In [ ]:
for layer in base_model.layers:
  layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(train_generator,epochs=25)

Epoch 1/25
351/351 [==============================] - 221s 595ms/step - loss: 3.3036 - accuracy: 0.1894
Epoch 2/25
351/351 [==============================] - 208s 593ms/step - loss: 2.0423 - accuracy: 0.2111
Epoch 3/25
351/351 [==============================] - 206s 586ms/step - loss: 1.8897 - accuracy: 0.2304
Epoch 4/25
351/351 [==============================] - 205s 583ms/step - loss: 1.8657 - accuracy: 0.2431
Epoch 5/25
351/351 [==============================] - 208s 593ms/step - loss: 1.8578 - accuracy: 0.2440
Epoch 6/25
351/351 [==============================] - 207s 591ms/step - loss: 1.8469 - accuracy: 0.2476
Epoch 7/25
351/351 [==============================] - 207s 588ms/step - loss: 1.8471 - accuracy: 0.2440
Epoch 8/25
351/351 [==============================] - 210s 597ms/step - loss: 1.8473 - accuracy: 0.2456
Epoch 9/25
351/351 [==============================] - 207s 590ms/step - loss: 1.8359 - accuracy: 0.2488
Epoch 10/25
351/351 [==============================] - 205s 584m

In [ ]:
model.save('F:\\Army University Level Term\\Level 4 Term 1\\All LAB\Ann lab\\Model\\ResNet50_baseline.h5')

Accuracy:  0.25345438718795776


In [ ]:
plt.plot(history.history['accuracy'])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
print("Accuracy: ",max(history.history['accuracy']))
plt.show()
     

In [ ]:
# Tesing
import tensorflow as tf
import numpy as np
from PIL import Image

# Load the pre-trained model
model = tf.keras.models.load_model('F:\\Army University Level Term\\Level 4 Term 1\\All LAB\Ann lab\\Model\\ResNet50_baseline.h5')

In [ ]:
# Define the classes
classes = ['angry', 'disgust', 'fear', 'happy', 'joiariya', 'meghna', 'netural', 'nitul', 'rafat', 'rasa', 'romiz', 'sad', 'surprise']

In [ ]:
# Load the image to be tested
image = Image.open('/content/Final datateset for Image/validation/fear/10267.jpg')

# Convert the image to RGB format
image = image.convert('RGB')

# Resize the image to the same size used for training the model
image = image.resize((224, 224))

# Convert the image to a numpy array
image_array = np.asarray(image)

# Normalize the pixel values to be between 0 and 1
image_array = image_array / 255.0

# Add a batch dimension to the image array
image_array = np.expand_dims(image_array, axis=0)

In [ ]:
# Predict the class of the image using the pre-trained model
predictions = model.predict(image_array)

1/1 [==============================] - 2s 2s/step


In [ ]:
# Get the index of the predicted class
predicted_class_index = np.argmax(predictions, axis=1)[0]

# Get the name of the predicted class
predicted_class = classes[predicted_class_index]

# Print the predicted class
print('The predicted class is:', predicted_class)

The predicted class is: rafat
